**整理数据**

In [5]:
import pandas as pd
from pathlib import Path
import json
import jieba
results = []

for fp in Path("/Users/wanghuan/Projects/simple-stt/outputs-yx1").glob("*.json"):
    with open(fp, "r") as f:
        d = json.load(f)
        
    text = "".join([r["text"] for r in d])
    results.append([fp.stem, text])

actuals = pd.read_excel("/Users/wanghuan/Projects/simple-stt/data-16k/groundtruth.xlsx")
actuals = actuals.rename(columns = {'filename':'fname'}).copy()
actuals['fname'] = actuals['fname'].str.replace(".pcm", "").copy()
    
preds = pd.DataFrame(results, columns = ['fname','pred']).copy()
merged = actuals.merge(preds, on='fname', how='inner')

def remove_punctuation(text):
    text = str(text)
    import re
    return re.sub(r'[^\w\s]', '', text)

def jieba_cut(text):
    text = str(text)
    return " ".join(jieba.cut(text))

merged['pred'] = merged['pred'].apply(remove_punctuation)
merged['target'] = merged['target'].apply(remove_punctuation)
merged['pred-cut'] = merged['pred'].apply(jieba_cut)
merged['target-cut'] = merged['target'].apply(jieba_cut)

/var/folders/9z/tc5kc_854l56plrg75dhnr7r0000gn/T/ipykernel_9886/3095364852.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  actuals['fname'] = actuals['fname'].str.replace(".pcm", "").copy()


**计算指标**

In [8]:
import jiwer
import numpy as np
wers = []
cers = []
for i, row in merged.iterrows():
    try:
        wer = jiwer.wer(row['pred-cut'], row['target-cut'])
    except:
        wer = 1
    wers.append(wer)
    try:
        cer = jiwer.cer(row['pred'], row['target'])
    except:
        cer = 1
    cers.append(cer)
    

merged['wer'] = wers
merged['cer'] = cers
print(f"Average WER: {np.mean(wers)}")
print(f"Average CER: {np.mean(cers)}")

Average WER: 0.506331945294862
Average CER: 0.4999720523340568


**Bad Case分析**

In [96]:
from IPython.display import Audio, display
import soundfile
badcase = merged[merged.wer > 0.7].fname.sample(1).iloc[0]
display(Audio(f"../data-16k/{badcase}.wav",autoplay=True,rate=16000))
record = merged[merged.fname == badcase]
print("Ground Truth:", record.iloc[0].target)
print("Prediction:", record.iloc[0].pred)
print("WER:", record.iloc[0].wer)
print("CER:", record.iloc[0].cer)


Ground Truth: 喂啊你说有啊
Prediction: 广州show上午上午行上午十点要跟麦手开会的效果十点钟吧左对有啊我吃完我周五中午
WER: 0.92
CER: 0.95


**Good Case分析**

In [95]:
goodcase = merged[merged.wer < 0.3].fname.sample(1).iloc[0]
display(Audio(f"../data-16k/{goodcase}.wav",autoplay=True,rate=16000))
record = merged[merged.fname == goodcase]
print("Ground Truth:", record.iloc[0].target)
print("Prediction:", record.iloc[0].pred)
print("WER:", record.iloc[0].wer)
print("CER:", record.iloc[0].cer)

Ground Truth: 嗯你说有啊
Prediction: 嗯你说有啊
WER: 0.0
CER: 0.0
